## Nessa aula vamos aprender como trabalhar em detalhes com séries temporais

1. Uma série temporal pode depender de outra série?
    1. Se pedender como trabalhamos com essa dependência?

2. Como é feita a avaliação de uma série temporal?

<img src="wfcv.png" />

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
import plotly.graph_objs as go

import optuna
import shap 

from sklearn.metrics import mean_squared_error

### Aqui vamos tentar prever o preço das ações da Petrobrás

In [2]:
preco_pb = pd.read_csv('PETR4.SA.csv')
del preco_pb['Volume']
preco_pb.head()

,Date,Open,High,Low,Close,Adj Close
0,2015-07-02,12.35,12.53,12.16,12.35,11.457355
1,2015-07-03,12.25,12.31,11.68,11.76,10.909998
2,2015-07-06,11.51,11.73,11.28,11.51,10.678070
3,2015-07-07,11.36,11.85,10.82,11.77,10.919277
4,2015-07-08,11.62,11.99,11.33,11.46,10.631682


### Vamos assumir que o preço da ação da empresa seja influenciado pelo preço do petroleo 

In [3]:
preco_petro = pd.read_csv('CL=F.csv')
del preco_petro['Volume']
preco_petro.head()

,Date,Open,High,Low,Close,Adj Close
0,2015-07-02,56.869999,57.950001,56.500000,56.930000,56.930000
1,2015-07-05,NaN,NaN,NaN,NaN,NaN
2,2015-07-06,54.900002,55.340000,52.410000,52.529999,52.529999
3,2015-07-07,52.750000,53.430000,50.580002,52.330002,52.330002
4,2015-07-08,52.910000,52.959999,50.910000,51.650002,51.650002


### E o preço do real será que tem alguma influência?

In [4]:
preco_real = pd.read_csv('BR=F.csv')
del preco_real['Volume']
preco_real.head()

,Date,Open,High,Low,Close,Adj Close
0,2015-07-02,0.31890,0.31890,0.31890,0.31890,0.31890
1,2015-07-05,NaN,NaN,NaN,NaN,NaN
2,2015-07-06,0.31460,0.31460,0.31460,0.31460,0.31460
3,2015-07-07,0.31035,0.31200,0.30980,0.31035,0.31035
4,2015-07-08,0.30675,0.31075,0.30585,0.30675,0.30675


### Vamos começar com alguns merges?

In [5]:
pb_petro = preco_pb.merge(preco_petro, on = 'Date', how = 'left')
pb_petro.head()

,Date,Open_x,High_x,Low_x,Close_x,Adj Close_x,Open_y,High_y,Low_y,Close_y,Adj Close_y
0,2015-07-02,12.35,12.53,12.16,12.35,11.457355,56.869999,57.950001,56.500000,56.930000,56.930000
1,2015-07-03,12.25,12.31,11.68,11.76,10.909998,NaN,NaN,NaN,NaN,NaN
2,2015-07-06,11.51,11.73,11.28,11.51,10.678070,54.900002,55.340000,52.410000,52.529999,52.529999
3,2015-07-07,11.36,11.85,10.82,11.77,10.919277,52.750000,53.430000,50.580002,52.330002,52.330002
4,2015-07-08,11.62,11.99,11.33,11.46,10.631682,52.910000,52.959999,50.910000,51.650002,51.650002


In [6]:
pb_petro_real = pb_petro.merge(preco_real, on = 'Date', how = 'left')
pb_petro_real.head()

,Date,Open_x,High_x,Low_x,Close_x,Adj Close_x,Open_y,High_y,Low_y,Close_y,Adj Close_y,Open,High,Low,Close,Adj Close
0,2015-07-02,12.35,12.53,12.16,12.35,11.457355,56.869999,57.950001,56.500000,56.930000,56.930000,0.31890,0.31890,0.31890,0.31890,0.31890
1,2015-07-03,12.25,12.31,11.68,11.76,10.909998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-07-06,11.51,11.73,11.28,11.51,10.678070,54.900002,55.340000,52.410000,52.529999,52.529999,0.31460,0.31460,0.31460,0.31460,0.31460
3,2015-07-07,11.36,11.85,10.82,11.77,10.919277,52.750000,53.430000,50.580002,52.330002,52.330002,0.31035,0.31200,0.30980,0.31035,0.31035
4,2015-07-08,11.62,11.99,11.33,11.46,10.631682,52.910000,52.959999,50.910000,51.650002,51.650002,0.30675,0.31075,0.30585,0.30675,0.30675


### Renomear as colunas para facilitar nossa vida

In [7]:
nomes={ 'Date' : 'Data',
        "Open_x": "Preco_Abertura_Petrobras", 
        "High_x": "Preco_Alta_Petrobras",
        "Low_x": "Preco_Baixo_Petrobras",
        "Close_x": "Preco_Fechamento_Petrobras",
        "Adj Close_x" : "Preco_Corrigido_Petrobras",
        "Open_y": "Preco_Abertura_Petróleo", 
        "High_y": "Preco_Alta_Petróleo",
        "Low_y": "Preco_Baixo_Petróleo",
        "Close_y": "Preco_Fechamento_Petróleo",
        "Adj Close_y" : "Preco_Corrigido_Petróleo",
        "Open": "Preco_Abertura_Real", 
        "High": "Preco_Alta_Real",
        "Low": "Preco_Baixo_Real",
        "Close": "Preco_Fechamento_Real",
        "Adj Close" : "Preco_Corrigido_Real"}

pb_petro_real.rename(columns = nomes, inplace = True)
pb_petro_real.head()

,Data,Preco_Abertura_Petrobras,Preco_Alta_Petrobras,Preco_Baixo_Petrobras,Preco_Fechamento_Petrobras,Preco_Corrigido_Petrobras,Preco_Abertura_Petróleo,Preco_Alta_Petróleo,Preco_Baixo_Petróleo,Preco_Fechamento_Petróleo,Preco_Corrigido_Petróleo,Preco_Abertura_Real,Preco_Alta_Real,Preco_Baixo_Real,Preco_Fechamento_Real,Preco_Corrigido_Real
0,2015-07-02,12.35,12.53,12.16,12.35,11.457355,56.869999,57.950001,56.500000,56.930000,56.930000,0.31890,0.31890,0.31890,0.31890,0.31890
1,2015-07-03,12.25,12.31,11.68,11.76,10.909998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-07-06,11.51,11.73,11.28,11.51,10.678070,54.900002,55.340000,52.410000,52.529999,52.529999,0.31460,0.31460,0.31460,0.31460,0.31460
3,2015-07-07,11.36,11.85,10.82,11.77,10.919277,52.750000,53.430000,50.580002,52.330002,52.330002,0.31035,0.31200,0.30980,0.31035,0.31035
4,2015-07-08,11.62,11.99,11.33,11.46,10.631682,52.910000,52.959999,50.910000,51.650002,51.650002,0.30675,0.31075,0.30585,0.30675,0.30675


In [8]:
pb_petro_real.dtypes

Data                           object
Preco_Abertura_Petrobras      float64
Preco_Alta_Petrobras          float64
Preco_Baixo_Petrobras         float64
Preco_Fechamento_Petrobras    float64
Preco_Corrigido_Petrobras     float64
Preco_Abertura_Petróleo       float64
Preco_Alta_Petróleo           float64
Preco_Baixo_Petróleo          float64
Preco_Fechamento_Petróleo     float64
Preco_Corrigido_Petróleo      float64
Preco_Abertura_Real           float64
Preco_Alta_Real               float64
Preco_Baixo_Real              float64
Preco_Fechamento_Real         float64
Preco_Corrigido_Real          float64
dtype: object

In [9]:
#Lembre de sempre definir a coluna que contém a data como datetime
pb_petro_real['Data'] = pd.to_datetime(pb_petro_real['Data'])

pb_petro_real.dtypes

Data                          datetime64[ns]
Preco_Abertura_Petrobras             float64
Preco_Alta_Petrobras                 float64
Preco_Baixo_Petrobras                float64
Preco_Fechamento_Petrobras           float64
Preco_Corrigido_Petrobras            float64
Preco_Abertura_Petróleo              float64
Preco_Alta_Petróleo                  float64
Preco_Baixo_Petróleo                 float64
Preco_Fechamento_Petróleo            float64
Preco_Corrigido_Petróleo             float64
Preco_Abertura_Real                  float64
Preco_Alta_Real                      float64
Preco_Baixo_Real                     float64
Preco_Fechamento_Real                float64
Preco_Corrigido_Real                 float64
dtype: object

In [10]:
## separar a variável alvo

y = pb_petro_real['Preco_Corrigido_Petrobras']
del pb_petro_real['Preco_Corrigido_Petrobras']

colunas = pb_petro_real.select_dtypes(exclude=['datetime64']).columns.tolist()

In [15]:
janela=30
for i in range (0, len(colunas)):
    pb_petro_real[str(colunas[i]+'_Media')] = pb_petro_real[str(colunas[i])].\
            rolling(window=janela).mean().round(4)

for i in range (0, len(colunas)):
    pb_petro_real[str(colunas[i]+'_%')] = pb_petro_real[str(colunas[i])].pct_change(janela-1).round(4)

In [20]:
pb_petro_real[29:32].T

,29,30,31
Data,2015-08-13 00:00:00,2015-08-14 00:00:00,2015-08-17 00:00:00
Preco_Abertura_Petrobras,9.91,9.61,9.2
Preco_Alta_Petrobras,9.97,9.68,9.36
Preco_Baixo_Petrobras,9.5,9.3,9.12
Preco_Fechamento_Petrobras,9.5,9.3,9.15
Preco_Abertura_Petróleo,43.34,42.23,42.18
Preco_Alta_Petróleo,43.72,42.96,42.69
Preco_Baixo_Petróleo,41.91,41.35,41.64
Preco_Fechamento_Petróleo,42.23,42.5,41.87
Preco_Corrigido_Petróleo,42.23,42.5,41.87


In [21]:
for i in range (0, len(colunas)):
    for j in range (int(i+1), len(colunas)):

        pb_petro_real['Correlacao_'+colunas[i]+colunas[j]] = \
            pb_petro_real[colunas[i]].rolling(window=janela).corr(pb_petro_real[colunas[j]]).round(4)
                

In [28]:
pb_petro_real[29:32].T

,29,30,31
Data,2015-08-13 00:00:00,2015-08-14 00:00:00,2015-08-17 00:00:00
Preco_Abertura_Petrobras,9.91,9.61,9.2
Preco_Alta_Petrobras,9.97,9.68,9.36
Preco_Baixo_Petrobras,9.5,9.3,9.12
Preco_Fechamento_Petrobras,9.5,9.3,9.15
...,...,...,...
Correlacao_Preco_Alta_RealPreco_Fechamento_Real,NaN,NaN,0.9945
Correlacao_Preco_Alta_RealPreco_Corrigido_Real,NaN,NaN,0.9945
Correlacao_Preco_Baixo_RealPreco_Fechamento_Real,NaN,NaN,0.9935
Correlacao_Preco_Baixo_RealPreco_Corrigido_Real,NaN,NaN,0.9935


In [29]:
colunas2 = pb_petro_real.select_dtypes(exclude=['datetime64']).columns.tolist()

In [30]:
for i in range (0, len(colunas2)):
    pb_petro_real[colunas2[i]] = pb_petro_real[colunas2[i]].shift(periods=janela)

In [31]:
pd.options.display.max_rows = None
pb_petro_real[29:32].T

,29,30,31
Data,2015-08-13,2015-08-14 00:00:00,2015-08-17 00:00:00
Preco_Abertura_Petrobras,NaT,12.35,12.25
Preco_Alta_Petrobras,NaT,12.53,12.31
Preco_Baixo_Petrobras,NaT,12.16,11.68
Preco_Fechamento_Petrobras,NaT,12.35,11.76
Preco_Abertura_Petróleo,NaT,56.87,NaN
Preco_Alta_Petróleo,NaT,57.95,NaN
Preco_Baixo_Petróleo,NaT,56.5,NaN
Preco_Fechamento_Petróleo,NaT,56.93,NaN
Preco_Corrigido_Petróleo,NaT,56.93,NaN


In [32]:
## Lembrar de remover colunas com alta correlação
# Matriz de Correlação
corr_matrix = pb_petro_real.corr().abs()
corr_matrix

,Preco_Abertura_Petrobras,Preco_Alta_Petrobras,Preco_Baixo_Petrobras,Preco_Fechamento_Petrobras,Preco_Abertura_Petróleo,Preco_Alta_Petróleo,Preco_Baixo_Petróleo,Preco_Fechamento_Petróleo,Preco_Corrigido_Petróleo,Preco_Abertura_Real,...,Correlacao_Preco_Abertura_RealPreco_Alta_Real,Correlacao_Preco_Abertura_RealPreco_Baixo_Real,Correlacao_Preco_Abertura_RealPreco_Fechamento_Real,Correlacao_Preco_Abertura_RealPreco_Corrigido_Real,Correlacao_Preco_Alta_RealPreco_Baixo_Real,Correlacao_Preco_Alta_RealPreco_Fechamento_Real,Correlacao_Preco_Alta_RealPreco_Corrigido_Real,Correlacao_Preco_Baixo_RealPreco_Fechamento_Real,Correlacao_Preco_Baixo_RealPreco_Corrigido_Real,Correlacao_Preco_Fechamento_RealPreco_Corrigido_Real
Preco_Abertura_Petrobras,1.000000,0.999363,0.998994,0.998407,0.586942,0.591968,0.568908,0.582803,0.582803,0.315660,...,0.039449,0.193761,0.031173,0.031173,0.221150,0.041442,0.041442,0.187906,0.187906,NaN
Preco_Alta_Petrobras,0.999363,1.000000,0.998735,0.999107,0.584659,0.590268,0.566425,0.581009,0.581009,0.321645,...,0.042097,0.198187,0.026190,0.026190,0.223139,0.044127,0.044127,0.192739,0.192739,NaN
Preco_Baixo_Petrobras,0.998994,0.998735,1.000000,0.999276,0.589234,0.594271,0.571898,0.585596,0.585596,0.307286,...,0.034949,0.191592,0.035352,0.035352,0.219289,0.036874,0.036874,0.185288,0.185288,NaN
Preco_Fechamento_Petrobras,0.998407,0.999107,0.999276,1.000000,0.586594,0.592144,0.569105,0.583514,0.583514,0.313876,...,0.037937,0.194847,0.028542,0.028542,0.221441,0.039951,0.039951,0.189221,0.189221,NaN
Preco_Abertura_Petróleo,0.586942,0.584659,0.589234,0.586594,1.000000,0.997860,0.988309,0.994179,0.994179,0.200439,...,0.000091,0.141335,0.622879,0.622879,0.088429,0.013238,0.013238,0.084906,0.084906,NaN
Preco_Alta_Petróleo,0.591968,0.590268,0.594271,0.592144,0.997860,1.000000,0.988455,0.996636,0.996636,0.186820,...,0.008595,0.152696,0.614143,0.614143,0.097504,0.004363,0.004363,0.096690,0.096690,NaN
Preco_Baixo_Petróleo,0.568908,0.566425,0.571898,0.569105,0.988309,0.988455,1.000000,0.993552,0.993552,0.220090,...,0.009558,0.130381,0.648358,0.648358,0.076221,0.022817,0.022817,0.071388,0.071388,NaN
Preco_Fechamento_Petróleo,0.582803,0.581009,0.585596,0.583514,0.994179,0.996636,0.993552,1.000000,1.000000,0.202238,...,0.000158,0.143695,0.635195,0.635195,0.088253,0.013350,0.013350,0.085804,0.085804,NaN
Preco_Corrigido_Petróleo,0.582803,0.581009,0.585596,0.583514,0.994179,0.996636,0.993552,1.000000,1.000000,0.202238,...,0.000158,0.143695,0.635195,0.635195,0.088253,0.013350,0.013350,0.085804,0.085804,NaN
Preco_Abertura_Real,0.315660,0.321645,0.307286,0.313876,0.200439,0.186820,0.220090,0.202238,0.202238,1.000000,...,0.247858,0.296485,0.570915,0.570915,0.219150,0.259878,0.259878,0.345674,0.345674,NaN


In [33]:
# Selecionar os elementos da parte superior da diagonal
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper

,Preco_Abertura_Petrobras,Preco_Alta_Petrobras,Preco_Baixo_Petrobras,Preco_Fechamento_Petrobras,Preco_Abertura_Petróleo,Preco_Alta_Petróleo,Preco_Baixo_Petróleo,Preco_Fechamento_Petróleo,Preco_Corrigido_Petróleo,Preco_Abertura_Real,...,Correlacao_Preco_Abertura_RealPreco_Alta_Real,Correlacao_Preco_Abertura_RealPreco_Baixo_Real,Correlacao_Preco_Abertura_RealPreco_Fechamento_Real,Correlacao_Preco_Abertura_RealPreco_Corrigido_Real,Correlacao_Preco_Alta_RealPreco_Baixo_Real,Correlacao_Preco_Alta_RealPreco_Fechamento_Real,Correlacao_Preco_Alta_RealPreco_Corrigido_Real,Correlacao_Preco_Baixo_RealPreco_Fechamento_Real,Correlacao_Preco_Baixo_RealPreco_Corrigido_Real,Correlacao_Preco_Fechamento_RealPreco_Corrigido_Real
Preco_Abertura_Petrobras,NaN,0.999363,0.998994,0.998407,0.586942,0.591968,0.568908,0.582803,0.582803,0.315660,...,0.039449,0.193761,0.031173,0.031173,0.221150,0.041442,0.041442,0.187906,0.187906,NaN
Preco_Alta_Petrobras,NaN,NaN,0.998735,0.999107,0.584659,0.590268,0.566425,0.581009,0.581009,0.321645,...,0.042097,0.198187,0.026190,0.026190,0.223139,0.044127,0.044127,0.192739,0.192739,NaN
Preco_Baixo_Petrobras,NaN,NaN,NaN,0.999276,0.589234,0.594271,0.571898,0.585596,0.585596,0.307286,...,0.034949,0.191592,0.035352,0.035352,0.219289,0.036874,0.036874,0.185288,0.185288,NaN
Preco_Fechamento_Petrobras,NaN,NaN,NaN,NaN,0.586594,0.592144,0.569105,0.583514,0.583514,0.313876,...,0.037937,0.194847,0.028542,0.028542,0.221441,0.039951,0.039951,0.189221,0.189221,NaN
Preco_Abertura_Petróleo,NaN,NaN,NaN,NaN,NaN,0.997860,0.988309,0.994179,0.994179,0.200439,...,0.000091,0.141335,0.622879,0.622879,0.088429,0.013238,0.013238,0.084906,0.084906,NaN
Preco_Alta_Petróleo,NaN,NaN,NaN,NaN,NaN,NaN,0.988455,0.996636,0.996636,0.186820,...,0.008595,0.152696,0.614143,0.614143,0.097504,0.004363,0.004363,0.096690,0.096690,NaN
Preco_Baixo_Petróleo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.993552,0.993552,0.220090,...,0.009558,0.130381,0.648358,0.648358,0.076221,0.022817,0.022817,0.071388,0.071388,NaN
Preco_Fechamento_Petróleo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.202238,...,0.000158,0.143695,0.635195,0.635195,0.088253,0.013350,0.013350,0.085804,0.085804,NaN
Preco_Corrigido_Petróleo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.202238,...,0.000158,0.143695,0.635195,0.635195,0.088253,0.013350,0.013350,0.085804,0.085804,NaN
Preco_Abertura_Real,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.247858,0.296485,0.570915,0.570915,0.219150,0.259878,0.259878,0.345674,0.345674,NaN


In [34]:
# Selecionar as colunas para deletar
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [35]:
pb_petro_real = pb_petro_real.drop(columns=to_drop)

In [36]:
pb_petro_real['Dia_do_Ano'] = pb_petro_real.Data.apply(lambda x: x.dayofyear)
pb_petro_real['Semana_do_Ano'] = pb_petro_real.Data.apply(lambda x: x.weekofyear)

In [37]:
pb_petro_real['Preco_Acao_Fechamento'] = y
pb_petro_real.shape

(1244, 31)

In [38]:
data = pb_petro_real['Data']
del pb_petro_real['Data']

In [39]:
pb_petro_real = pb_petro_real.interpolate(method='linear', limit_direction='forward', axis=1)
pb_petro_real['Data'] = data

In [40]:
pb_petro_real.dropna(how='any', inplace = True)

In [41]:
pb_petro_real.shape

(1211, 31)

In [42]:

pb_petro_real.head()

,Preco_Abertura_Petrobras,Preco_Abertura_Petróleo,Preco_Abertura_Real,Preco_Abertura_Petrobras_%,Preco_Abertura_Petróleo_%,Preco_Baixo_Petróleo_%,Preco_Abertura_Real_%,Correlacao_Preco_Abertura_PetrobrasPreco_Alta_Petrobras,Correlacao_Preco_Abertura_PetrobrasPreco_Baixo_Petrobras,Correlacao_Preco_Abertura_PetrobrasPreco_Fechamento_Petrobras,...,Correlacao_Preco_Fechamento_PetróleoPreco_Corrigido_Petróleo,Correlacao_Preco_Abertura_RealPreco_Alta_Real,Correlacao_Preco_Abertura_RealPreco_Baixo_Real,Correlacao_Preco_Abertura_RealPreco_Fechamento_Real,Correlacao_Preco_Alta_RealPreco_Baixo_Real,Correlacao_Preco_Fechamento_RealPreco_Corrigido_Real,Dia_do_Ano,Semana_do_Ano,Preco_Acao_Fechamento,Data
30,12.35,56.869999,0.318900,9.346144,18.373388,27.400632,36.427876,45.455120,54.482364,63.509608,...,171.836536,180.863780,189.891024,198.918268,207.945512,216.972756,226.0,33.0,8.627806,2015-08-14
31,12.25,20.277778,28.305556,36.333333,44.361111,52.388889,60.416667,68.444444,76.472222,84.500000,...,180.833333,188.861111,196.888889,204.916667,212.944444,220.972222,229.0,34.0,8.488647,2015-08-17
32,11.51,54.900002,0.314600,9.502016,18.689432,27.876848,37.064264,46.251680,55.439096,64.626512,...,174.875504,184.062920,193.250336,202.437752,211.625168,220.812584,230.0,34.0,8.340212,2015-08-18
33,11.36,52.750000,0.310350,9.537936,18.765522,27.993108,37.220694,46.448280,55.675866,64.903452,...,175.634484,184.862070,194.089656,203.317242,212.544828,221.772414,231.0,34.0,8.108282,2015-08-19
34,11.62,52.910000,0.306750,9.574480,18.842210,28.109940,37.377670,46.645400,55.913130,65.180860,...,176.393620,185.661350,194.929080,204.196810,213.464540,222.732270,232.0,34.0,8.061894,2015-08-20


In [43]:
pb_petro_real['Data'].min()

Timestamp('2015-08-14 00:00:00')

In [44]:
pb_petro_real['Data'].max()

Timestamp('2020-07-01 00:00:00')

In [45]:
pb_petro_real['Ano'] = pb_petro_real['Data'].dt.year
pb_petro_real['Mes'] = pb_petro_real['Data'].dt.month
pb_petro_real['Dia'] = pb_petro_real['Data'].dt.day

In [46]:
#Listas para salvar as métricas e parâmetros
parametros = []
metricas = []

val = []
pred = []

data = []

In [47]:
data1 = ['2015-10-01', '2015-11-01', '2015-12-01', '2016-01-01',
         '2016-02-01',  '2016-03-01', '2016-04-01', 
        '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01'
        , '2016-09-01', '2016-10-01', '2016-11-01']

data2 = ['2018-10-01', '2018-11-01', '2018-12-01','2019-01-01',
         '2019-02-01', '2019-03-01', '2019-04-01', 
        '2019-05-01',  '2019-06-01',  '2019-07-01', '2019-08-01'
         '2019-09-01', '2019-10-01', '2019-11-01']

data3 = [ '2018-11-01', '2018-12-01', '2019-01-01',
        '2019-02-01', '2019-03-01', '2019-04-01', '2019-05-01',
        '2019-06-01', '2019-07-10', '2019-08-01'
        ,'2019-09-01', '2019-10-01',  '2019-11-01', '2019-12-01']

data4 = ['2018-12-01', '2019-01-01', '2019-02-01',
        '2019-03-01', '2019-04-01', '2019-05-01',
        '2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01'
        ,'2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01']

In [48]:
len(data1)

14

In [49]:
for i in range (0, 10):

    #treino
    treino = pb_petro_real.loc[(pb_petro_real['Data']>=data1[i]) & (pb_petro_real['Data'] < data2[i])]
    y_treino = treino['Preco_Acao_Fechamento'].values
    data_treino = treino ['Data']
    del treino['Preco_Acao_Fechamento']
    del treino ['Data']

    #validação
    validacao = pb_petro_real.loc[(pb_petro_real['Data']>=data2[i]) & (pb_petro_real['Data'] < data3[i])]
    y_validacao = validacao['Preco_Acao_Fechamento'].values
    data_validacao = validacao['Data']
    del validacao['Preco_Acao_Fechamento']
    del validacao ['Data']
    
    #teste
    teste = pb_petro_real.loc[(pb_petro_real['Data']>=data3[i]) & (pb_petro_real['Data'] < data4[i])]
    y_teste = teste['Preco_Acao_Fechamento'].values
    data_teste = teste ['Data']
    del teste['Preco_Acao_Fechamento']
    del teste ['Data']

    
#    if teste.shape[0] >= 5:
#        if validacao.shape[0] >= 5:
    lgb_train = lgb.Dataset(treino.values, y_treino)
    lgb_eval = lgb.Dataset(validacao.values, y_validacao, reference=lgb_train)

    def objective(trial):

            # 2. Suggest values of the hyperparameters using a trial object.
            param = {
                'objective': 'regression',
                'metric': 'mse',
                'boosting_type': trial.suggest_categorical("boosting_type", ["gbdt", "dart"]),
                'num_leaves': trial.suggest_int('num_leaves', 2, 256),
                'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
                'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
                'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                'max_depth': trial.suggest_int('max_depth', 5, 100),
            }

            gbm = lgb.train(param, lgb_train)
            y_pred = gbm.predict(validacao.values)
            rmse = mean_squared_error(y_validacao, y_pred) ** 0.5
            return rmse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=30)

    final_model = lgb.LGBMRegressor(**study.best_params)
    y_pred2 = final_model.fit(treino.values, y_treino).predict(teste)

    rmse = mean_squared_error(y_teste ,y_pred2) ** 0.5
    print(rmse)

    parametros.append(study.best_params)
    metricas.append(rmse)
    val.append(y_validacao)
    pred.append(y_pred2)
    data.append(data_validacao)

[I 2020-07-02 22:02:13,860] Finished trial#0 with value: 6.54442792999266 with parameters: {'boosting_type': 'gbdt', 'num_leaves': 251, 'feature_fraction': 0.41558816535668436, 'bagging_fraction': 0.8419383314379572, 'bagging_freq': 2, 'min_child_samples': 79, 'max_depth': 65}. Best is trial#0 with value: 6.54442792999266.
[I 2020-07-02 22:02:14,260] Finished trial#1 with value: 8.486006438091518 with parameters: {'boosting_type': 'dart', 'num_leaves': 187, 'feature_fraction': 0.8495279856758382, 'bagging_fraction': 0.6541765435503608, 'bagging_freq': 3, 'min_child_samples': 46, 'max_depth': 56}. Best is trial#0 with value: 6.54442792999266.
[I 2020-07-02 22:02:14,592] Finished trial#2 with value: 6.819767520537571 with parameters: {'boosting_type': 'gbdt', 'num_leaves': 92, 'feature_fraction': 0.9619286916498669, 'bagging_fraction': 0.5655404932193057, 'bagging_freq': 7, 'min_child_samples': 98, 'max_depth': 59}. Best is trial#0 with value: 6.54442792999266.
[I 2020-07-02 22:02:14,901

6.012971854088638


[I 2020-07-02 22:02:30,193] Finished trial#0 with value: 3.5380577798787796 with parameters: {'boosting_type': 'gbdt', 'num_leaves': 165, 'feature_fraction': 0.7237996028328795, 'bagging_fraction': 0.4859173196049187, 'bagging_freq': 5, 'min_child_samples': 64, 'max_depth': 5}. Best is trial#0 with value: 3.5380577798787796.
[I 2020-07-02 22:02:30,613] Finished trial#1 with value: 3.8168061207206483 with parameters: {'boosting_type': 'gbdt', 'num_leaves': 203, 'feature_fraction': 0.8271542537566003, 'bagging_fraction': 0.6142121643853315, 'bagging_freq': 5, 'min_child_samples': 56, 'max_depth': 87}. Best is trial#0 with value: 3.5380577798787796.
[I 2020-07-02 22:02:31,083] Finished trial#2 with value: 6.653012252430298 with parameters: {'boosting_type': 'dart', 'num_leaves': 215, 'feature_fraction': 0.6759578412435157, 'bagging_fraction': 0.9397175389936202, 'bagging_freq': 2, 'min_child_samples': 46, 'max_depth': 9}. Best is trial#0 with value: 3.5380577798787796.
[I 2020-07-02 22:02

1.4082695913615793


[I 2020-07-02 22:02:49,243] Finished trial#0 with value: 4.059582342761472 with parameters: {'boosting_type': 'dart', 'num_leaves': 27, 'feature_fraction': 0.6300490323437284, 'bagging_fraction': 0.549884857092795, 'bagging_freq': 7, 'min_child_samples': 39, 'max_depth': 31}. Best is trial#0 with value: 4.059582342761472.
[I 2020-07-02 22:02:49,770] Finished trial#1 with value: 1.4185619206325735 with parameters: {'boosting_type': 'gbdt', 'num_leaves': 26, 'feature_fraction': 0.9928294941102391, 'bagging_fraction': 0.8289393945514452, 'bagging_freq': 1, 'min_child_samples': 41, 'max_depth': 45}. Best is trial#1 with value: 1.4185619206325735.
[I 2020-07-02 22:02:50,154] Finished trial#2 with value: 1.577177677350907 with parameters: {'boosting_type': 'gbdt', 'num_leaves': 193, 'feature_fraction': 0.8633376273523636, 'bagging_fraction': 0.6857974385748715, 'bagging_freq': 1, 'min_child_samples': 86, 'max_depth': 28}. Best is trial#1 with value: 1.4185619206325735.
[I 2020-07-02 22:02:50

...

2.6781429763922238


[I 2020-07-02 22:04:30,326] Finished trial#0 with value: 3.734818764715996 with parameters: {'boosting_type': 'gbdt', 'num_leaves': 50, 'feature_fraction': 0.4374496576640474, 'bagging_fraction': 0.8329208650551128, 'bagging_freq': 2, 'min_child_samples': 48, 'max_depth': 21}. Best is trial#0 with value: 3.734818764715996.
[I 2020-07-02 22:04:30,620] Finished trial#1 with value: 5.414110299541302 with parameters: {'boosting_type': 'dart', 'num_leaves': 231, 'feature_fraction': 0.4434532363661977, 'bagging_fraction': 0.6469234198132965, 'bagging_freq': 1, 'min_child_samples': 60, 'max_depth': 35}. Best is trial#0 with value: 3.734818764715996.
[I 2020-07-02 22:04:30,870] Finished trial#2 with value: 6.550563048943886 with parameters: {'boosting_type': 'dart', 'num_leaves': 247, 'feature_fraction': 0.5769876670429583, 'bagging_fraction': 0.5391039848380919, 'bagging_freq': 2, 'min_child_samples': 99, 'max_depth': 58}. Best is trial#0 with value: 3.734818764715996.
[I 2020-07-02 22:04:31,

2.4445208036355655


[I 2020-07-02 22:04:39,017] Finished trial#0 with value: 4.071019050294456 with parameters: {'boosting_type': 'dart', 'num_leaves': 112, 'feature_fraction': 0.954602635547015, 'bagging_fraction': 0.7408043383384235, 'bagging_freq': 6, 'min_child_samples': 99, 'max_depth': 87}. Best is trial#0 with value: 4.071019050294456.
[I 2020-07-02 22:04:39,483] Finished trial#1 with value: 1.3619419408727151 with parameters: {'boosting_type': 'gbdt', 'num_leaves': 68, 'feature_fraction': 0.9939114890219161, 'bagging_fraction': 0.9506203616587963, 'bagging_freq': 4, 'min_child_samples': 28, 'max_depth': 98}. Best is trial#1 with value: 1.3619419408727151.
[I 2020-07-02 22:04:40,030] Finished trial#2 with value: 4.912821485270778 with parameters: {'boosting_type': 'dart', 'num_leaves': 41, 'feature_fraction': 0.7862531403638682, 'bagging_fraction': 0.7947572954394668, 'bagging_freq': 4, 'min_child_samples': 11, 'max_depth': 42}. Best is trial#1 with value: 1.3619419408727151.
[I 2020-07-02 22:04:40

1.0794897857965882


In [50]:
metricas

[6.012971854088638,
 1.4082695913615793,
 8.020857867409791,
 6.637847960384765,
 5.454539821857647,
 2.2255773728131403,
 1.3380753495753088,
 2.6781429763922238,
 2.4445208036355655,
 1.0794897857965882]

In [51]:
np.argmin(metricas)

9

In [61]:
print(len(parametros))
print(parametros[np.argmin(metricas)])

10
{'boosting_type': 'gbdt', 'num_leaves': 68, 'feature_fraction': 0.9939114890219161, 'bagging_fraction': 0.9506203616587963, 'bagging_freq': 4, 'min_child_samples': 28, 'max_depth': 98}


In [96]:
final_model = lgb.LGBMRegressor(**parametros[np.argmin(metricas)])
final_model.fit(treino.values, y_treino)
y_val_pred = final_model.predict(validacao)
y_test_pred = final_model.predict(teste)


In [99]:
y_real = np.concatenate([y_treino, y_validacao, y_teste])
y_prd = np.concatenate([y_treino, y_val_pred, y_test_pred])
data = np.concatenate([data_treino, data_validacao, data_teste])

In [101]:
plt.figure(figsize=(20,10))
plt.plot(data,y_real)
plt.plot(y_prd)

[<matplotlib.lines.Line2D at 0x1cfc036b0d0>]

ValueError: view limit minimum -36865.05 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units